In [1]:
from ROOT import TCanvas
from ROOT import TGraph, TGraphErrors
from ROOT import gStyle
from array import array
import numpy as np
import math

xcanvas = 1000
ycanvas = 800

c1 = TCanvas( 'c1', 'Pendulum Amplitude vs. Time', 0, 0, xcanvas, ycanvas )
c1.SetGridx()
c1.SetGridy()
c1.GetFrame().SetFillColor( 21 )
c1.GetFrame().SetBorderMode(-1 )
c1.GetFrame().SetBorderSize( 5 )
c2 = TCanvas( 'c2', 'Pendulum Period vs. Time', 0, 0, xcanvas, ycanvas )
c2.SetGridx()
c2.SetGridy()
c2.GetFrame().SetFillColor( 21 )
c2.GetFrame().SetBorderMode(-1 )
c2.GetFrame().SetBorderSize( 5 )

Welcome to JupyROOT 6.18/00


In [2]:
# Initial Conditions
length = 9.807
method = 2 # 1 = Euler, 2 = Verlet
theta0 = 45.0
theta = theta0*math.pi/180.0
omega = 0.0

# Other constants
gravity = 9.807 # gravitional acceleration
g_over_L = gravity/length
time = 0.0
irev = 0.0
tau = 0.001 

Pi = math.pi

In [3]:
accel = -g_over_L*math.sin(theta)
theta_old = theta - omega*tau + 0.5*tau*tau*accel
print (accel, theta, theta_old)

-0.7071067811865475 0.7853981633974483 0.7853978098440577


In [4]:
tau = 0.0001 # timestep in seconds
nStep = 10000 # number of steps

t_plot = array('d')
th_plot = array('d')
period = array('d')
trev = array('d')

In [5]:
for iStep in range(1,nStep+1):
    t_plot.append(time)
    th_plot.append(theta*180/Pi)
    time = time + tau
    accel = -g_over_L*math.sin(theta)
    
    if (method == 1):
        theta_old = theta
        theta = theta + tau*omega
        omega = omega + tau*accel
    else:
        theta_new = 2*theta - theta_old + tau*tau*accel
        theta_old = theta
        theta = theta_new
    
    if (theta*theta_old < 0):
        print ("Turning point at time t = %f" % time)
        if (irev == 0):
            time_old = time
        else:
            period.append(2.0*(time - time_old))
            trev.append(time)
            time_old = time
        irev = irev + 1

In [6]:
nPeriod = int(irev - 1)
print (nPeriod)

-1


In [7]:
gr = TGraph(nStep,t_plot,th_plot)
grr = TGraph(nPeriod,trev,period)

TypeError: none of the 11 overloaded methods succeeded. Full details:
  TGraph::TGraph() =>
    takes at most 0 arguments (3 given)
  TGraph::TGraph(int n) =>
    takes at most 1 arguments (3 given)
  TGraph::TGraph(const TGraph& gr) =>
    takes at most 1 arguments (3 given)
  TGraph::TGraph(const TVectorT<float>& vx, const TVectorT<float>& vy) =>
    takes at most 2 arguments (3 given)
  TGraph::TGraph(const TVectorT<double>& vx, const TVectorT<double>& vy) =>
    takes at most 2 arguments (3 given)
  TGraph::TGraph(const TH1* h) =>
    takes at most 1 arguments (3 given)
  TGraph::TGraph(const TF1* f, const char* option = "") =>
    takes at most 2 arguments (3 given)
  TGraph::TGraph(const char* filename, const char* format = "%lg %lg", const char* option = "") =>
    could not convert argument 1 (bad argument type for built-in operation)
  TGraph::TGraph(int n, const int* x, const int* y) =>
    could not convert argument 2
  TGraph::TGraph(int n, const float* x, const float* y) =

In [ ]:
gr.SetMarkerColor(4)
grr.SetMarkerColor(4)
gr.SetMarkerStyle(21)
grr.SetMarkerStyle(22)
gr.SetTitle("Pendulum Amplitude vs. Time")
gr.GetXaxis().SetTitle("Time")
gr.GetYaxis().SetTitle("Amplitude")
grr.SetTitle("Pendulum Period vs. Time")
grr.GetXaxis().SetTitle("Time")
grr.GetYaxis().SetTitle("Period")

c1.cd()
gr.Draw("AP")
c1.Draw()

In [ ]:
c2.cd()
grr.Draw("AP")
c2.Draw()

In [ ]:
AvePeriod = 0.0
ErrorBar = 0.0
for i in range(1,nPeriod+1):
    AvePeriod = AvePeriod + period[i-1]
AvePeriod = AvePeriod/nPeriod
for i in range(1,nPeriod+1):
    ErrorBar = ErrorBar + (period[i-1]-AvePeriod)*(period[i-1]-AvePeriod)
ErrorBar = math.sqrt(ErrorBar/(nPeriod*(nPeriod-1)))
print("Average Period = %f +/- %f" % (AvePeriod,ErrorBar))
    

In [ ]:
t0=theta0*Pi/180.0
t_infinite = 2.0*Pi/math.sqrt(g_over_L)*(1.0+t0*t0/16.0+math.pow(t0,4)*11.0/3072.0+math.pow(t0,6)*173.0/737280.0)
error_infinite = 2.0*Pi/math.sqrt(g_over_L)*math.pow(t0,8)*22931.0/1321205760.0

In [ ]:
print("Infinite series prediction = %f +/- %f" % (t_infinite,error_infinite))

In [ ]:
x = array('d')
y = array('d')
ex = array('d')
ey = array('d')
npoints = 2
x.append(1)
x.append(2)
ex.append(0)
ex.append(0)
y.append(AvePeriod)
y.append(t_infinite)
ey.append(ErrorBar)
ey.append(error_infinite)

c3 = TCanvas("c3","c3",100,0,800,600)
gr3 = TGraphErrors(npoints,x,y,ex,ey)
gr3.SetMarkerStyle(21)
gr3.SetTitle("Theory vs. Simulation")
gr3.GetXaxis().SetTitle("1=Sim,2=Theory")
gr3.GetYaxis().SetTitle("Average Period (s)")
gr3.Draw("AP")
c3.Draw()